# 05: Report Export

Export tables and figures for the final report.


In [1]:
import sys
from pathlib import Path

# Add src to path - robust path resolution for notebooks
# In Jupyter, cwd is usually the project root, not notebooks/
current_dir = Path.cwd()

# Check if we're in notebooks/ directory or at project root
if current_dir.name == 'notebooks':
    project_root = current_dir.parent
elif (current_dir / 'config.yml').exists():
    # We're already at project root
    project_root = current_dir
else:
    # Try to find project root by looking for config.yml
    project_root = current_dir
    while project_root != project_root.parent:
        if (project_root / 'config.yml').exists():
            break
        project_root = project_root.parent

sys.path.insert(0, str(project_root))

import pandas as pd
from src.analytics.performance_metrics import calculate_all_metrics
from src.viz.plots import (
    plot_cumulative_returns,
    plot_drawdown,
    plot_rolling_metric,
    plot_correlation_matrix,
)
from src.analytics.risk import calculate_drawdown_series
from src.analytics.performance_metrics import calculate_rolling_sharpe
from src.utils.io import save_dataframe, save_figure
from src.utils.paths import get_data_interim_path


In [2]:
# Load data
interim_path = get_data_interim_path()
returns_df = pd.read_csv(interim_path / "returns_merged.csv", parse_dates=["date"])
returns_df = returns_df.set_index("date")

varbx_returns = returns_df["return_varbx"]
sp500_returns = returns_df["return_sp500"]
agg_returns = returns_df["return_agg"]


## Export Performance Metrics Table


In [3]:
# Calculate and export metrics table
varbx_metrics = calculate_all_metrics(varbx_returns)
sp500_metrics = calculate_all_metrics(sp500_returns)
agg_metrics = calculate_all_metrics(agg_returns)

metrics_df = pd.DataFrame({
    "VARBX": varbx_metrics,
    "S&P 500": sp500_metrics,
    "AGG": agg_metrics
})

save_dataframe(metrics_df, "performance_metrics", formats=["csv", "latex"])
print("Exported performance metrics table")


ImportError: Missing optional dependency 'Jinja2'. DataFrame.style requires jinja2. Use pip or conda to install Jinja2.

## Export Figures


In [ ]:
# Export cumulative returns plot
returns_dict = {
    "VARBX": varbx_returns,
    "S&P 500": sp500_returns,
    "AGG": agg_returns
}
fig = plot_cumulative_returns(returns_dict)
save_figure(fig, "cumulative_returns", formats=["png", "pdf"])
print("Exported cumulative returns figure")


In [ ]:
# Export drawdown plot
varbx_drawdown = calculate_drawdown_series(varbx_returns)
fig = plot_drawdown(varbx_drawdown)
save_figure(fig, "drawdown", formats=["png", "pdf"])
print("Exported drawdown figure")


In [ ]:
# Export correlation matrix
returns_only = returns_df[["return_varbx", "return_sp500", "return_agg"]]
fig = plot_correlation_matrix(returns_only)
save_figure(fig, "correlation_matrix", formats=["png", "pdf"])
print("Exported correlation matrix figure")
